# Exporting ImageNet Inception

In this notebook, we'll show how to export the [pre-trained Imagenet Inception model](https://tfhub.dev/google/imagenet/inception_v3/classification/3) for serving.

First, we'll install the required packages:

In [0]:
!pip install tensorflow==1.14.* tensorflow-hub==0.6.* boto3==1.*

Next, we'll download the model from TensorFlow Hub and export it for serving:

In [0]:
import time
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.python.saved_model.signature_def_utils_impl import predict_signature_def

export_dir = "export/" + str(time.time()).split('.')[0]
builder = tf.saved_model.builder.SavedModelBuilder(export_dir)

with tf.Session(graph=tf.Graph()) as sess:
    module = hub.Module("https://tfhub.dev/google/imagenet/inception_v3/classification/3")

    input_params = module.get_input_info_dict()
    image_input = tf.placeholder(
        name="images", dtype=input_params["images"].dtype, shape=input_params["images"].get_shape()
    )
    
    sess.run([tf.global_variables_initializer(), tf.tables_initializer()])

    classes = module(image_input)
    signature = predict_signature_def(inputs={"images": image_input}, outputs={"classes": classes})

    builder.add_meta_graph_and_variables(
        sess, ["serve"], signature_def_map={"predict": signature}, strip_default_attrs=True
    )

builder.save()

## Upload the model to AWS

Cortex loads models from AWS, so we need to upload the exported model.

Set these variables to configure your AWS credentials and model upload path:

In [0]:
AWS_ACCESS_KEY_ID = "" #@param {type:"string"}
AWS_SECRET_ACCESS_KEY = "" #@param {type:"string"}
S3_UPLOAD_PATH = "s3://my-bucket/image-classifier/inception" #@param {type:"string"}

import sys
import re

if AWS_ACCESS_KEY_ID == "":
    print("\033[91m{}\033[00m".format("ERROR: Please set AWS_ACCESS_KEY_ID"), file=sys.stderr)

elif AWS_SECRET_ACCESS_KEY == "":
    print("\033[91m{}\033[00m".format("ERROR: Please set AWS_SECRET_ACCESS_KEY"), file=sys.stderr)

else:
    try:
        bucket, key = re.match("s3://(.+?)/(.+)", S3_UPLOAD_PATH).groups()
    except:
        print("\033[91m{}\033[00m".format("ERROR: Invalid s3 path (should be of the form s3://my-bucket/path/to/file)"), file=sys.stderr)

Upload the model to S3:

In [0]:
import os
import boto3

s3 = boto3.client("s3", aws_access_key_id=AWS_ACCESS_KEY_ID, aws_secret_access_key=AWS_SECRET_ACCESS_KEY)

for dirpath, _, filenames in os.walk("export"):
    for filename in filenames:
        filepath = os.path.join(dirpath, filename)
        filekey = os.path.join(key, filepath[len("export/"):])
        print("Uploading s3://{}/{}...".format(bucket, filekey), end = '')
        s3.upload_file(filepath, bucket, filekey)
        print(" ✓")

print("\nUploaded model export directory to " + S3_UPLOAD_PATH)